In [43]:
import pyomo.environ as pe
import pyomo.opt as po
from pyomo.opt import SolverFactory
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import time
import math
import gurobipy

from collections import defaultdict
import os

In [44]:
solver = po.SolverFactory('gurobi')

In [45]:
# Path definition
Inputpath = "../input/"
Outputpath = "../output/numerical_results/"

In [46]:
# Basic fixed parameters
L = 26 # 196/7.5
T = 2 # Evacuation time window
G = 3.7   # cost of charging capacity installed at each node, million $/thousand cars = thousand $/ car
#C = 4    # capacity of each arc, in thousands of cars per hour
B = 10    # budget, million $

df_arc = pd.read_excel(os.path.join(Inputpath,"arcs_sonoma_15min.xlsx"), header = 0)
df_node = pd.read_excel(os.path.join(Inputpath,"nodes_sonoma_15min.xlsx"), header = 0)
df_char_damFac = pd.read_csv(os.path.join(Inputpath, "charging_cap_damFac_3hrs.csv"), header = 0)
df_road_damFac = pd.read_csv(os.path.join(Inputpath, "road_cap_damFac_3hrs.csv"), header = 0)
df_S_nodes = pd.read_excel(os.path.join(Inputpath,"safe_nodes_15min.xlsx"), header = 0)
df_S_nodes = df_S_nodes[df_S_nodes['Safe'] == 1]

In [47]:
# random scenario probabilities and
cases_num = max(df_char_damFac['Scenario'])
prob_dict = dict()
for k in range(cases_num):
    prob_dict[k] = df_char_damFac.groupby(['Scenario']).first()['Probability'][k+1]

### Define Graph

### Sets

In [48]:
## SETS

# The set of nodes in the safety area, put the components of dataframe in a set
N_s = set()
for i in range(len(df_S_nodes)):
    N_s.add(df_S_nodes.iloc[i]['Nodes'])

# Set of all nodes
N = set()
for i in range(len(df_node)):
    N.add(df_node['Nodes'][i])

A = set()
# edge_info = dict()
road_cap = dict()

for i in range(len(df_arc)):
    node0 = df_arc['From'][i]
    node1 = df_arc['To'][i]
    A.add((node0, node1))
    road_cap[((node0, node1))] = df_arc['Capacity'][i]

In [49]:
## STOCHASTIC PARAMETERS c: road capacity ##
def c_random(A, df_road_damFac, cases_num, road_cap, T):

    #c = dict()
    c_tuple = dict()

    gk = df_road_damFac.groupby(['Scenario'])

    for k in range(cases_num):
        sce_sub = gk.get_group(k+1).copy()
#         sce_sub = df_road_damFac[df_road_damFac['Scenario'] == k+1]
        #c[k] = {}
        for r in range(sce_sub.shape[0]):
            curr_edge = sce_sub.iloc[r]

            for t in range(4*T+1):
                if math.isnan(curr_edge['Time']):
                    c_tuple[((int(curr_edge['From']), int(curr_edge['To'])), k),t] = road_cap[(int(curr_edge['From']), int(curr_edge['To']))]
                elif t/4 < curr_edge['Time']:
                    c_tuple[((int(curr_edge['From']), int(curr_edge['To'])), k),t] = road_cap[(int(curr_edge['From']), int(curr_edge['To']))]
                else:
                    c_tuple[((int(curr_edge['From']), int(curr_edge['To'])), k),t] = road_cap[(int(curr_edge['From']), int(curr_edge['To']))] * 0

            # expand road capacity by a constant number to adapt the Houston metro area population
            # c[k][(int(curr_edge['From']), int(curr_edge['To']))] = curr_edge['DamFac'] * road_cap[(int(curr_edge['From']), int(curr_edge['To']))]
            # c_tuple[(int(curr_edge['From']), int(curr_edge['To'])), k] = curr_edge['DamFac'] * road_cap[(int(curr_edge['From']), int(curr_edge['To']))] #* 2

    return c_tuple

In [50]:
## STOCHASTIC PARAMETERS xi: charging capacity factors ##
def s_random(df_char_damFac, cases_num, T):

    xi_s = dict()

    gk = df_char_damFac.groupby(['Scenario'])

    for k in range(cases_num):
        sce_sub = gk.get_group(k+1).copy()
#         sce_sub = df_char_damFac[df_char_damFac['Scenario'] == k+1]

        for r in range(sce_sub.shape[0]):
            curr_node = sce_sub.iloc[r]
            for t in range(4*T+1):
                if math.isnan(curr_node['Time']):
                    xi_s[(int(curr_node['Nodes']), k),t] = 1
                elif t/4 < curr_node['Time']:
                    xi_s[(int(curr_node['Nodes']), k),t] = 1
                else:
                    xi_s[(int(curr_node['Nodes']), k),t] = 0

    return xi_s

In [51]:
road_ca = c_random(A, df_road_damFac, cases_num, road_cap, T) # road capacity random parameter
xi_s = s_random(df_char_damFac, cases_num, T) # charging capacity random parameter

In [52]:
Vi = defaultdict(set)
Vo = defaultdict(set)
for (i,j) in A:
    Vi[j].add(i) # incoming set
    Vo[i].add(j) # outgoing set

### Model

In [53]:
# two stage stochastic model we built
def primal_run_res(L,T,cases_num,N,N_s,A,car_num,road_ca,xi_s,Vi,Vo,B,G,Outputpath,out_folder_name, out_folder, Total_cars, Cat=None,p=None):
    start = time.time() #record the start time
    model = pe.ConcreteModel()

    # Model settings
    model.Levels = range(L)
    model.Times = range(4*T+1)
    model.Cases = range(cases_num) # number of cases

    model.nodes = pe.Set(initialize=N) # N is the set of all nodes
    model.nodes_safe = pe.Set(initialize=N_s) # the set of all safe nodes
    model.edges = pe.Set(initialize=A) # the set of all edges

    # car number initialization
    model.car_num = pe.Param(model.Levels, model.nodes, initialize=car_num)
    # road capacity
    model.road_ca = pe.Param(model.edges, model.Cases, model.Times, initialize=road_ca)
    # charging capacity random damage factor
    model.char_damFac = pe.Param(model.nodes, model.Cases, model.Times, initialize=xi_s)
    # scenario probabilities
    model.prob = pe.Param(model.Cases, initialize=prob_dict)

    # define parameters of incoming and outgoing sets
    model.Vi = pe.Param(model.nodes, initialize=Vi, default=set(), within=pe.Any)
    model.Vo = pe.Param(model.nodes, initialize=Vo, default=set(), within=pe.Any)


    # VARIABLES
    # variable x_{l,(i,j),t,k}
    model.x = pe.Var(model.Levels, model.edges, model.Times, model.Cases, within=pe.NonNegativeReals) # stage 2
    # variable y_{l,(j,j),t,k}
    model.y = pe.Var(model.Levels, model.nodes, model.Times, model.Cases, within=pe.NonNegativeReals) # stage 2
    # variable z: number of cars at node j
    model.z = pe.Var(model.Levels, model.nodes, model.Times, model.Cases, within=pe.NonNegativeReals) # stage 2
    # variable s_{j,k}: number of chargers (in thousands)
    model.s = pe.Var(model.nodes, within=pe.NonNegativeReals) # stage 1

    # OBJECTIVE VALUE
    obj_func_value = 0
    total_budget_expr = 0

    # CONSTRAINT LISTS
    model.con_flow_bal = pe.ConstraintList()
    model.con_charge_limit = pe.ConstraintList()
    model.con_road_ca = pe.ConstraintList()
    model.budget_con = pe.ConstraintList()
    model.con_x_init = pe.ConstraintList()
    model.con_y_init = pe.ConstraintList()
    model.con_y = pe.ConstraintList() # constraint for y while l = 0
    model.con_z_init = pe.ConstraintList()

    # 1st stage constraint: BUDGET OF INVESTMENT
    total_budget_expr = sum(G*model.s[j] for j in model.nodes)
    model.budget_con.add(expr = (total_budget_expr <= B))


    # 2nd stage constraints
    for w in range(cases_num):

        # objective function value: sum(p_i * obj_i over i)
        obj_func_value += model.prob[w] * -sum( sum( sum(model.z[l,j,t,w] for j in model.nodes_safe) + sum(model.y[l,j,t,w] for j in model.nodes_safe) for l in model.Levels) for t in model.Times)

        # FLOW BALANCE constraints
        for j in model.nodes:
            for t in range(4*T):
                for l in model.Levels:
                    # first constraint
                    if l >= 1 and l <= L-2:
                        flow_in = sum(model.x[l,i,j,t,w] for i in model.Vi[j]) + model.y[l,j,t,w] + model.z[l,j,t,w]
                        flow_out = model.y[l+1,j,t+1,w] + model.z[l,j,t+1,w] + sum(model.x[l-1,j,k,t+1,w] for k in model.Vo[j] if k != j)
                    # second constraint: at min level
                    if l == 0:
                        flow_in = sum(model.x[l,i,j,t,w] for i in model.Vi[j]) + model.z[l,j,t,w]
                        flow_out = model.y[l+1,j,t+1,w] + model.z[l,j,t+1,w]
                    # third constraint: at max level
                    if l == L-1:
                        flow_in = model.y[l,j,t,w] + model.z[l,j,t,w]
                        flow_out = sum(model.x[l-1,j,k,t+1,w] for k in model.Vo[j] if k != j) + model.z[l,j,t+1,w]

                    model.con_flow_bal.add(expr = (flow_in == flow_out))


        # CHARGING LIMIT constraint
        for j in model.nodes:
            for t in model.Times:
                LHS = sum(model.y[l,j,t,w] for l in range(1,L))
                #RHS = model.s[j] * model.char_damFac[j,w]
                RHS = model.s[j] * model.char_damFac[j,w,t]
                model.con_charge_limit.add(expr = (LHS <= RHS))


        # ROAD CAPACITY constraint
        for (i,j) in model.edges:
            for t in model.Times:
                if i != j:
                    LHS = sum(model.x[l,i,j,t,w] for l in range(L-1))
                    #RHS = model.road_ca[i,j,w]
                    RHS = model.road_ca[i,j,w,t]
                    model.con_road_ca.add(expr = (LHS <= RHS))


        # x INITIALIZATION
        for (i,j) in model.edges:
            for w in model.Cases:
                for t in model.Times:
                    con_expr = (model.x[L-1,i,j,t,w] == 0) # cars cannot reach the next node with full charge at any time
                    model.con_x_init.add(expr = con_expr)
                for l in model.Levels:
                    con_expr = (model.x[l,i,j,0,w] == 0) # x initialization at time t = 0, no cars on arc when t=0
                    model.con_x_init.add(expr = con_expr)


        # y INITIALIZATION
        for j in model.nodes:
            for l in model.Levels:
                for w in model.Cases:
                    con_expr = (model.y[l,j,0,w] == 0) # no cars charging at t=0
                    model.con_y_init.add(expr = con_expr)


        # z initialization: cars ready to evacuate at t=0
        for j in model.nodes:
            for l in model.Levels:
                for w in model.Cases:
                    con_expr = (model.z[l,j,0,w] == model.car_num[l,j])
                    model.con_z_init.add(expr = con_expr)

        # constraint for y while l = 0: cannot have cars charging to level 0
        for j in model.nodes:
            for t in model.Times:
                for w in model.Cases:
                    y_con_expr = (model.y[0,j,t,w] == 0)
                    model.con_y.add(expr = y_con_expr)



    model.obj_max_reward = pe.Objective(sense=pe.minimize, expr = obj_func_value)


    result = solver.solve(model, tee=True)

    end = time.time()
    run_time = end - start
    print("execution time: " + str(end - start))


    # POST PROCESSING
    s_output = {'Nodes':[],'CharCap':[]}
    for j in model.nodes:
        s_output['Nodes'].append(j)
        s_output['CharCap'].append(model.s[j].value)
    df_s_output = pd.DataFrame(s_output)

    x_output = {'Arcs':[],'Levels':[],'Times':[],'Scenarios':[],'NumCars':[]}
    y_output = {'Nodes':[],'Levels':[],'Times':[],'Scenarios':[],'NumCars':[]}
    z_output = {'Nodes':[],'Levels':[],'Times':[],'Scenarios':[],'NumCars':[]}

    for (i,j) in model.edges:
        for l in model.Levels:
            for t in model.Times:
                for w in model.Cases:
                    x_output['Arcs'].append((i,j))
                    x_output['Levels'].append(l)
                    x_output['Times'].append(t)
                    x_output['Scenarios'].append(w)
                    x_output['NumCars'].append(model.x[l,i,j,t,w].value)

    for j in model.nodes:
        for l in model.Levels:
            for t in model.Times:
                for w in model.Cases:
                    y_output['Nodes'].append(j)
                    z_output['Nodes'].append(j)

                    y_output['Levels'].append(l)
                    z_output['Levels'].append(l)

                    y_output['Times'].append(t)
                    z_output['Times'].append(t)

                    y_output['Scenarios'].append(w)
                    z_output['Scenarios'].append(w)

                    y_output['NumCars'].append(model.y[l,j,t,w].value)
                    z_output['NumCars'].append(model.z[l,j,t,w].value)

    df_x_output = pd.DataFrame(x_output)
    df_y_output = pd.DataFrame(y_output)
    df_z_output = pd.DataFrame(z_output)

    # out_folder_name = "B" + str(B) + "T"  + str(T) + "_"+"high_battery" + "L" + str(L)
    # out_folder = os.path.join(Outputpath, "normal_speed", out_folder_name)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)

    s_outname = "s_values.csv"
    x_outname = "x_values.csv"
    y_outname = "y_values.csv"
    z_outname = "z_values.csv"

    df_s_output.to_csv(os.path.join(out_folder, s_outname),index=False)
    df_x_output.to_csv(os.path.join(out_folder, x_outname),index=False)
    df_y_output.to_csv(os.path.join(out_folder, y_outname),index=False)
    df_z_output.to_csv(os.path.join(out_folder, z_outname),index=False)

# only sum y and z
    num_evac = sum(model.prob[w] * sum(sum((model.y[l,j,4*T,w].value + model.z[l,j,4*T,w].value) for j in model.nodes_safe) for l in model.Levels) for w in model.Cases)
    # for all success EVs
    avg_time = {}
    avg_time_all = {}
    for w in model.Cases:
        suc_cars = sum(sum((model.y[l,j,4*T,w].value + model.z[l,j,4*T,w].value) for j in model.nodes_safe) for l in model.Levels)
        avg_time[w] = 0
        for t in model.Times:
            if t >= 1:
                avg_time[w] = avg_time[w] + t * (sum(sum((model.y[l,j,t,w].value + model.z[l,j,t,w].value) for j in model.nodes_safe) for l in model.Levels) - sum(sum((model.y[l,j,t-1,w].value + model.z[l,j,t-1,w].value) for j in model.nodes_safe) for l in model.Levels))
        avg_time_all[w] = (avg_time[w] + (4*T) * (Total_cars - suc_cars))/Total_cars
        avg_time[w] = avg_time[w] / suc_cars
        print(avg_time)

    avg_evac_time = sum((model.prob[w] * avg_time[w]) for w in model.Cases)
    avg_evac_time_all = sum((model.prob[w] * avg_time_all[w]) for w in model.Cases)
    # consider all EVs, the failure one accounts for 4 hrs in total

    return num_evac, avg_evac_time, avg_evac_time_all, run_time

In [54]:
# run the base case
# def base_study_even_run(L):
#     # Read corresponding csv for init
#     df_car_init = pd.read_csv(os.path.join(Inputpath,"CarNumberInitial_halfEV_L"+str(L-1)+"even"+".csv"),header=0)
#     # Convert dataframe to dictionary
#     car_num = dict()
#     print(df_car_init)
#     for line in range(len(df_car_init)):
#         car_num[(df_car_init['Levels'][line], df_car_init['Nodes'][line])] = df_car_init['CarNumbers'][line]
#     Total_cars = 73.203
#     num_evac,avg_evac_time,avg_evac_time_all,run_time = primal_run_res(L,T,cases_num,N,N_s,A,car_num,road_ca,xi_s,Vi,Vo,B,G,Outputpath,Total_cars)
#     return num_evac, avg_evac_time, avg_evac_time_all, run_time

In [55]:
# sensitivity analysis: budget and evacuation time window (change)
# budget range is [1,4,7,10,13,16,19]
budget_low = 1
budget_high = 22
budget_list = list(range(budget_low,budget_high,3))

# time window don't change
T_low = 2
T_high = 3
T_list = list(range(T_low,T_high,1))

# res_dic = {"B":[],"EvacTime":[],"NumEvac":[],"RunTime":[]}

# L = 14 # EV range: 13
file_names = ["even_battery", "high_battery", "low_battery"]
folder_names = ["even_battery_normal_speed", "high_battery_normal_speed", "low_battery_normal_speed"]

def main_sens_anal(L):
    # Read corresponding csv for init
    for i in range(len(file_names)):
        df_car_init = pd.read_csv(os.path.join(Inputpath, "car_charge_distribution","CarNumberInitial_halfEV_L"+str(L-1)+str(file_names[i])+".csv"),header=0)
        # Convert dataframe to dictionary
        car_num = dict()
        for line in range(len(df_car_init)):
            car_num[(df_car_init['Levels'][line], df_car_init['Nodes'][line])] = df_car_init['CarNumbers'][line]
        #Total number of cars
        Total_cars = 73.203
        res_dic = {"B":[],"EvacTime":[],"NumEvac":[],"Avg_evac_time":[],"Avg_evac_time_all":[],"RunTime":[]}

        # Loop over scenarios
        for B in budget_list:
            for T in T_list:
                road_ca = c_random(A, df_road_damFac, cases_num, road_cap, T) # road capacity random parameter
                xi_s = s_random(df_char_damFac, cases_num, T) # charging capacity random parameter
                name1 = "B" + str(B) + "T"  + str(T) + "_"+ str(file_names[i]) + "L" + str(L)
                name2 = os.path.join(Outputpath, folder_names[i], name1)
                num_evac,avg_evac_time,avg_evac_time_all,run_time = primal_run_res(L,T,cases_num,N,N_s,A,car_num,road_ca,xi_s,Vi,Vo,B,G,Outputpath,name1,name2,Total_cars)
                res_dic['B'].append(B)
                res_dic['EvacTime'].append(T)
                res_dic['NumEvac'].append(num_evac)
                res_dic['Avg_evac_time'].append(avg_evac_time)
                res_dic['Avg_evac_time_all'].append(avg_evac_time_all)
                res_dic['RunTime'].append(run_time)
        print(res_dic)

        df_res = pd.DataFrame(res_dic)

        out_folder_name = "B" + str(budget_low)+'_'+str(budget_high) + "T" + str(T_low) + "_" + str(T_high-1) + "_" + str(file_names[i]) + "_L"+str(L) + "sensitivity_analysis"
        out_folder = os.path.join(Outputpath, folder_names[i], out_folder_name)
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)

        res_filename = "num_evac" + "B" + str(budget_low)+'_'+str(budget_high) + "T" + str(T_low) + "_" + str(T_high-1)+ "L" + str(L) + "_sensitivity" + ".csv"
        df_res.to_csv(os.path.join(out_folder,res_filename),index=False)

In [56]:
main_sens_anal(27)

    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    data source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-22
Read LP format model from file /var/folders/39/jnzvpm091qz13hy6ywbpf89r0000gn/T/tmpsyv68ms4.pyomo.lp
Reading time = 3.23 seconds
x563787: 1705322 rows, 563787 columns, 2976077 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 1705322 rows, 563787 columns and 2976077 nonzeros
Model fingerprint: 0xd843eefc
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-05, 3e+00]
Pre

KeyboardInterrupt: 